<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/wisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyannote-audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.8 MB/s eta 0:00:00

In [1]:
import numpy as np
from pyannote.audio import Pipeline, Audio
from pyannote.core import Segment
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [2]:
!pip install moviepy

In [3]:
from moviepy.editor import VideoFileClip

  if event.key is 'enter':



In [5]:
video_path = "./9d4fc932d442ef2ffb17d0a8086390e6_1725887318814_0.mp4"
video_path = "./Sample_data_Hisajima-Mitsumura.mp4"
video_path = "./InterviewZero_2_audio_mask_改.mp4"
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

MoviePy - Writing audio in ./InterviewZero_2_audio_mask_改.wav


MoviePy - Done.


In [4]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jdwgb3v0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jdwgb3v0
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=de829baa1bb3d71e237e6b3267d5c4f99228a5c35213166762d170a6d0a01222
  Stored in directory: /tmp/pip-ephem-wheel-cache-tvfkvgrp/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [6]:
import whisper
import json, re

In [7]:
model = whisper.load_model("large")
_ = model.cuda()

100%|█████████████████████████████████████| 2.88G/2.88G [00:40<00:00, 75.7MiB/s]
  checkpoint = torch.load(fp, map_location=device)



In [53]:
result = model.transcribe(audio_path, verbose=True, fp16=False, language="ja", word_timestamps=True)

[00:04.700 --> 00:05.960] お疲れ様です
[00:08.260 --> 00:09.480] 聞こえますか
[00:14.180 --> 00:15.860] あれ 新さんの声聞こえないな
[00:15.860 --> 00:17.560] 今オンになってます
[00:18.600 --> 00:19.980] お疲れ様です
[00:20.620 --> 00:25.880] 新さんの声が届かないのでちょっとお待ちくださいませ
[00:26.340 --> 00:26.700] あれ
[00:28.380 --> 00:29.260] 来ました
[00:33.600 --> 00:35.760] こちらの声届いてます
[00:36.440 --> 00:38.160] 届いてます 聞こえてます
[00:38.160 --> 00:40.960] 私のやつは音声出るので
[00:41.580 --> 00:44.040] ちょっと一回僕マイクを
[00:51.600 --> 00:54.040] 今度聞こえなくなりました
[00:55.760 --> 00:56.200] ちょっと
[00:56.200 --> 00:58.500] 私はお二人の声
[00:58.500 --> 01:00.960] 今聞こえます
[01:02.080 --> 01:03.700] 僕の方がおかしいんだ
[01:06.920 --> 01:08.300] これでOKです
[01:09.340 --> 01:11.020] 全部許可したんだけど
[01:11.020 --> 01:11.960] まあいいよ
[01:11.960 --> 01:15.920] そしてどうしよう
[01:17.140 --> 01:19.940] 読書に関するインタビュー
[01:21.080 --> 01:21.560] まず
[01:22.540 --> 01:23.020] まず
[01:24.300 --> 01:25.840] アンケート
[01:26.680 --> 01:29.040] 説問とかありましたっけ
[01:29.620 --> 01:32.540] はい 説問3つ用意してまして
[01:33.240 --> 01:33.400] はい
[01:34.400 

In [54]:
whisper_result = {}
whisper_segment = []
for asr in result['segments']:
    if asr['end'] - asr['start'] >= 0.6 and len(asr['text']) >= 2:
        asr['text'] += '。'
        whisper_segment.append(asr)
whisper_result['segments'] = whisper_segment

In [55]:
segments = whisper_result['segments']
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
for seg in segments:
  count = len(prog.findall(seg['text']))
  if count > 0: print(str(seg['id']) + " : " + seg['text'])
  _seg = {'start': round(seg['start'], 2), 'end': round(seg['end'], 2), 'text': seg['text'], 'count': count}
  INTERVIEW.append(_seg)

5 : 新さんの声が届かないのでちょっとお待ちくださいませ。
31 : 電子書籍で読むジャンルって何でしょうか。
35 : 日本語のジャンルって読めばいいですか?。
36 : はい 日本語のジャンルって読めばいいですか?。
54 : それは中国語なんですか?。
57 : 中国語の歴史が長いじゃないですか。
59 : どこにあるかよく謎になっているじゃないですか。
73 : 教師とか出てくるじゃないですか。
78 : 映画化とかされてるんですか中国で。
81 : ちょっと待っててくださいね。
92 : これ日本語訳されてないんですか。
130 : どういうことなんですかね。
146 : これをいつも読んでるんですか?。
180 : 子供で生きてからですかね。読めなくなっちゃったのが。。
186 : ちょっとまだそこまでは行ってないですかね。。
197 : けどKindleだったら戻れるじゃないですか。
200 : あれ?Kindleって買うんですか?小説の中で。
206 : 2種類があるじゃないですか。
221 : Oasis軽いですか?重さって。
236 : 例えばこういった縦で読む人がいるじゃないですか。
246 : バッテリーどうなんですか?。
294 : なんでですか?。
315 : あれですか、Wi-Fiとか繋げる?。
325 : Amazonのアカウントがあるじゃないですか。
403 : 何が違うんですかね。
455 : 見てくださいみたいな形なので。
481 : 4冊で分かれているっていうやつもあったりするじゃないですか。
504 : いろんなポイントとかがあるじゃないですか。
534 : お子さんのお子さんは今お子さんがいくつですか?。
536 : おー中3の子にどんな本を読ませているんですか?。
589 : 小さなお子さんはいくつなんですか。
594 : え一緒に住まわれてるんですか。
604 : 礼拝からあけどめっちゃいいじゃないですか。
617 : 教学校もあったじゃないですか女の子と男の子が教学もあったんだけれどもうちの子が天性あまり得意じゃないからそして女の子はこういう感じで。
619 : 思考にするとライバルは半分減るじゃない ですかああああああ。
636 : 思考の方がどうなんですか 自由いう伸びの糸育っ。
650 : 進化こうしたんです

In [56]:
segments = whisper_result['segments']
embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device='cuda')
embeddings = np.zeros(shape=(len(segments), 192))

duration = video.duration
embeddings = []


for i, segment in enumerate(segments):
    audio = Audio()
    start = round(segment["start"], 2)
    end = round(min(duration, segment["end"]), 2)
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(audio_path, clip)
    print(waveform[0])
    _enbed = embedding_model(waveform[None])
    #print(_enbed[0])
    embeddings.append(_enbed[0])

embeddings = np.nan_to_num(embeddings)


  state_dict = torch.load(path, map_location=device)

  stats = torch.load(path, map_location=device)



tensor([ 0.0057, -0.0031, -0.0051,  ...,  0.0095,  0.0132,  0.0144])
tensor([-0.0002, -0.0002, -0.0003,  ..., -0.0031, -0.0013,  0.0018])
tensor([9.1553e-05, 1.5259e-04, 3.3569e-04,  ..., 1.0461e-01, 1.0471e-01,
        1.0391e-01])
tensor([ 0.1041,  0.1054,  0.1061,  ..., -0.0140, -0.0129, -0.0137])
tensor([0.0865, 0.0981, 0.1097,  ..., 0.0323, 0.0342, 0.0384])
tensor([-0.0270, -0.0233, -0.0243,  ...,  0.0224,  0.0206,  0.0096])
tensor([0.1836, 0.1375, 0.0862,  ..., 0.0499, 0.0262, 0.0026])
tensor([-9.1553e-05,  2.1362e-04, -9.1553e-05,  ...,  6.1646e-03,
         5.4016e-03,  5.4016e-03])
tensor([1.2207e-04, 6.1035e-05, 3.0518e-04,  ..., 3.3569e-02, 4.8401e-02,
        6.2683e-02])
tensor([0.0764, 0.0887, 0.0993,  ..., 0.0011, 0.0017, 0.0022])
tensor([ 3.6621e-04, -3.0518e-05, -2.4414e-04,  ...,  9.7565e-02,
         9.4696e-02,  8.9661e-02])
tensor([-0.0103, -0.0108, -0.0101,  ..., -0.0046, -0.0051, -0.0049])
tensor([ 0.0112,  0.0125,  0.0139,  ..., -0.0072, -0.0025,  0.0025])
tenso

In [57]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
speaker_count = 3

In [58]:
embeddings.shape

(1022, 192)

In [59]:
clustering_agg = AgglomerativeClustering(speaker_count).fit(embeddings)

In [60]:
clustering_kmeans = KMeans(n_clusters=speaker_count).fit(embeddings)

In [61]:
#clustering = clustering_agg
clustering = clustering_kmeans

labeled_segments = []
for label, segment in zip(clustering.labels_, whisper_result["segments"]):
    labeled_segments.append((label, segment["start"], segment["text"]))

speek = [0] * speaker_count
for s, (speaker, _, text) in enumerate(labeled_segments):
    speek[speaker] += INTERVIEW[s]["count"]
inteviewer_index = np.argmax(speek)


interivew = []
print(INTERVIEW)
for s, (speaker, _, text) in enumerate(labeled_segments):
    stime = INTERVIEW[s]["start"]
    SPEAKER = 'モニタ　　　　'
    if speaker == inteviewer_index:
      SPEAKER = 'インタビューア'
    #interivew.append(f"{SPEAKER} :  {stime} 〜 : 「{text}」")
    interivew.append(f"SPEAKER_{speaker} :  {stime} 〜 : 「{text}」")

[{'start': 4.7, 'end': 5.96, 'text': 'お疲れ様です。', 'count': 0}, {'start': 8.26, 'end': 9.48, 'text': '聞こえますか。', 'count': 0}, {'start': 14.18, 'end': 15.86, 'text': 'あれ 新さんの声聞こえないな。', 'count': 0}, {'start': 15.86, 'end': 17.56, 'text': '今オンになってます。', 'count': 0}, {'start': 18.6, 'end': 19.98, 'text': 'お疲れ様です。', 'count': 0}, {'start': 20.62, 'end': 25.88, 'text': '新さんの声が届かないのでちょっとお待ちくださいませ。', 'count': 1}, {'start': 28.38, 'end': 29.26, 'text': '来ました。', 'count': 0}, {'start': 33.6, 'end': 35.76, 'text': 'こちらの声届いてます。', 'count': 0}, {'start': 36.44, 'end': 38.16, 'text': '届いてます 聞こえてます。', 'count': 0}, {'start': 38.16, 'end': 40.96, 'text': '私のやつは音声出るので。', 'count': 0}, {'start': 41.58, 'end': 44.04, 'text': 'ちょっと一回僕マイクを。', 'count': 0}, {'start': 51.6, 'end': 54.04, 'text': '今度聞こえなくなりました。', 'count': 0}, {'start': 56.2, 'end': 58.5, 'text': '私はお二人の声。', 'count': 0}, {'start': 58.5, 'end': 60.96, 'text': '今聞こえます。', 'count': 0}, {'start': 62.08, 'end': 63.7, 'text': '僕の方がおかしいんだ。', 'count': 0}, {'start

In [62]:
for speek in interivew:
  print(speek)

SPEAKER_0 :  4.7 〜 : 「お疲れ様です。」
SPEAKER_1 :  8.26 〜 : 「聞こえますか。」
SPEAKER_1 :  14.18 〜 : 「あれ 新さんの声聞こえないな。」
SPEAKER_2 :  15.86 〜 : 「今オンになってます。」
SPEAKER_2 :  18.6 〜 : 「お疲れ様です。」
SPEAKER_2 :  20.62 〜 : 「新さんの声が届かないのでちょっとお待ちくださいませ。」
SPEAKER_1 :  28.38 〜 : 「来ました。」
SPEAKER_2 :  33.6 〜 : 「こちらの声届いてます。」
SPEAKER_0 :  36.44 〜 : 「届いてます 聞こえてます。」
SPEAKER_0 :  38.16 〜 : 「私のやつは音声出るので。」
SPEAKER_2 :  41.58 〜 : 「ちょっと一回僕マイクを。」
SPEAKER_0 :  51.6 〜 : 「今度聞こえなくなりました。」
SPEAKER_0 :  56.2 〜 : 「私はお二人の声。」
SPEAKER_0 :  58.5 〜 : 「今聞こえます。」
SPEAKER_2 :  62.08 〜 : 「僕の方がおかしいんだ。」
SPEAKER_2 :  66.92 〜 : 「これでOKです。」
SPEAKER_1 :  69.34 〜 : 「全部許可したんだけど。」
SPEAKER_1 :  71.02 〜 : 「まあいいよ。」
SPEAKER_1 :  71.96 〜 : 「そしてどうしよう。」
SPEAKER_0 :  77.14 〜 : 「読書に関するインタビュー。」
SPEAKER_0 :  84.3 〜 : 「アンケート。」
SPEAKER_0 :  86.68 〜 : 「説問とかありましたっけ。」
SPEAKER_2 :  89.62 〜 : 「はい 説問3つ用意してまして。」
SPEAKER_2 :  94.4 〜 : 「まずこれが読書って言っても。」
SPEAKER_2 :  96.18 〜 : 「電子書籍に関する。」
SPEAKER_0 :  99.08 〜 : 「インタビューになってますと。」
SPEAKER_2 :  100.9 〜 : 「電子書籍で読むジャンルって何でしょうか。」
SPEAKER

In [63]:
InterReview_json = {"movie_name": video_path}
speak_json = []
for s, (speaker, _, text) in enumerate(labeled_segments):
    _speak = {"start": f'{INTERVIEW[s]["start"]:.2f}s', "end": f'{INTERVIEW[s]["end"]:.2f}s', 'speaker': f'SPEAKER_{speaker}', 'speak': text}
    speak_json.append(_speak)
InterReview_json['InterReview'] = speak_json

json_path = '.'.join(video_path.split('.')[:-1]) + '.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(InterReview_json, f, indent=4, ensure_ascii=False)

In [64]:
speak_json

[{'start': '4.70s',
  'end': '5.96s',
  'speaker': 'SPEAKER_0',
  'speak': 'お疲れ様です。'},
 {'start': '8.26s',
  'end': '9.48s',
  'speaker': 'SPEAKER_1',
  'speak': '聞こえますか。'},
 {'start': '14.18s',
  'end': '15.86s',
  'speaker': 'SPEAKER_1',
  'speak': 'あれ 新さんの声聞こえないな。'},
 {'start': '15.86s',
  'end': '17.56s',
  'speaker': 'SPEAKER_2',
  'speak': '今オンになってます。'},
 {'start': '18.60s',
  'end': '19.98s',
  'speaker': 'SPEAKER_2',
  'speak': 'お疲れ様です。'},
 {'start': '20.62s',
  'end': '25.88s',
  'speaker': 'SPEAKER_2',
  'speak': '新さんの声が届かないのでちょっとお待ちくださいませ。'},
 {'start': '28.38s',
  'end': '29.26s',
  'speaker': 'SPEAKER_1',
  'speak': '来ました。'},
 {'start': '33.60s',
  'end': '35.76s',
  'speaker': 'SPEAKER_2',
  'speak': 'こちらの声届いてます。'},
 {'start': '36.44s',
  'end': '38.16s',
  'speaker': 'SPEAKER_0',
  'speak': '届いてます 聞こえてます。'},
 {'start': '38.16s',
  'end': '40.96s',
  'speaker': 'SPEAKER_0',
  'speak': '私のやつは音声出るので。'},
 {'start': '41.58s',
  'end': '44.04s',
  'speaker': 'SPEAKER_2',
  'sp